Assignment Overview
This assignment will test your ability to code your own version of ridge-regularized regression in Python. Ridge Regression offers a way to mitigate some of the weaknesses of Least Squares Linear Regression to build more robust models. This assignment draws upon and presupposes the knowledge found in the lectures for Module 13.

The assignment also builds upon the work performed in the module 13 assignment "Linear Regression - Least Squares". The data used will be the same. Though the last assignment tested your ability to read data into Pandas from a .csv. Those fundamental processes will not be directly tested here.

This assignment is designed to build your familiarity and comfort coding in Python while also helping you review key topics from each module. As you progress through the assignment, answers will get increasingly complex. It is important that you adopt a data scientist's mindset when completing this assignment. Remember to run your code from each cell before submitting your assignment. Running your code beforehand will notify you of errors and give you a chance to fix your errors before submitting. You should view your Vocareum submission as if you are delivering a final project to your manager or client.

Vocareum Tips

Do not add arguments or options to functions unless you are specifically asked to. This will cause an error in Vocareum.
Do not use a library unless you are expicitly asked to in the question.
You can download the Grading Report after submitting the assignment. This will include feedback and hints on incorrect questions.


Ridge Regression and Bayesian Methods
For this assignment, we will use a regression model on a housing price dataset to predict the price of a house based on its living area above the ground. More information about this dataset can be found here.

Before coding an algorithm, we will take a look at our data using Python's pandas. For visualizations, we'll use matplotlib. Let's import the necessary libraries and load the datasets by using the pandas pd.read_csv() function.



In [ ]:
### Import the necessary modules

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20.0, 10.0)  

### Read in the data
tr_path = 'data/train.csv'

data = pd.read_csv(tr_path)  

We begin by performing some basic exploratory data analysis by using the function head() and the attribute columns.

In [ ]:
data.head()


Question 1:
5 points

Which column has the most "null" values? Assign name of the column as string to ans1a. How many nulls are in that column? Assign number as int to ans1b.

NOTE: To find the column that has the higher number of null values, use the idxmin() function.



In [ ]:
### GRADED

### YOUR SOLUTION HERE
ans1a = None
ans1b = None

### BEGIN SOLUTION
ans1a = data.count().idxmin()
ans1b = 1453
### END SOLUTION

Next, let's plot the relationship between our variables of interest: the price for each house and the above ground living area in square feet.

We can do so by creating a scatter plot using matplotlib.

In [ ]:
data.plot('GrLivArea', 'SalePrice', kind = 'scatter', marker = 'x')

Question 2:
5 points

Create a subset of our dataframe below containing only the "Street" and "Alley" columns from the data. Assign the new dataframe to 'ans2'

In [ ]:
### GRADED

### YOUR SOLUTION HERE
ans2 = None

### BEGIN SOLUTION
cols = ['Street','Alley']
ans2 = data[cols]
### END SOLUTION



Coding Ridge Regression
Preprocessing
Before implementing ridge regression, it is important to mean-center our target variable and mean-center and standardize observations. We will do this by using the following formulas:

Mean Center Target
ycent=y0−y¯
 
Standardize Observations
Xstd=X0−X¯sX
 
Where  X¯  is the sample mean of X and  sX  is the sample standard deviation of X.

NOTE: The sample standard deviation should be calculated with 0 "Delta Degrees of Freedom"

Back to top

Question 3:
5 points

Why are the centering/standardization transformations described above important for ridge regression?

a) Regression works best when values are unitless
b) The transformations makes the regression more interpretable
c) Ridge penalizes large coefficients; the transformations make the coefficients of similar scales
d) It isn't important
Assign character associated with your choice as a string to ans3.

In [ ]:
### GRADED

### YOUR ANSWER BELOW

ans3 = None

### BEGIN SOLUTION
ans3 = 'c'
### END SOLUTION


Question 4:
10 points

Standardized values can be calculated using the following formula:

Xstd=X0−X¯sX.
 
Define a function "standardize" that accepts, as input a list of numbers and returns a list where those values have been standardized. Please use the NumPy function std() for calculating standard deviation

NOTE: The sample standard deviation should be calculated with 0 "Delta Degrees of Freedom".
If your answer does not match the example answer, check the default degrees of freedom in your standard deviation function.

In [ ]:
### GRADED

### YOUR ANSWER BELOW

def standardize( num_list):
    """
    Standardize the given list of numbers
    
    Positional arguments:
        num_list -- a list of numbers
    
    Example:
        num_list = [1,2,3,3,4,4,5,5,5,5,5]
        nl_std = standardize(num_list)
        print(np.round(nl_std,2))
        #--> np.array([-2.11, -1.36, -0.61, -0.61,  
                0.14,  0.14,  0.88,  0.88,  0.88,
                0.88,  0.88])
    
    NOTE: the sample standard deviation should be calculated with 0 "Delta Degrees of Freedom"
    """
    
    return 

### BEGIN SOLUTION
def standardize(num_list):
    
    # Calculate standard deviation and mean
    std = np.std(num_list)
    mean = np.mean(num_list)
    
    # Implement equation
    toRet = [(x-mean)/std for x in num_list]
    
    return toRet
### END SOLUTION

Below we will create a function which will preprocess our data by performing:

mean subtraction from  y ,
dimension standardization for  x .
The formulas to Mean Center Target and Standardize Observations are given above.

NOTE: The sample standard deviation should be calculated with 0 "Delta Degrees of Freedom"
If your answer does not match the example answer, check the default degrees of freedom in your standard deviation function.

Back to top

Question 5:
10 points

Code a function called "preprocess_for_regularization" that accepts, as input, the DataFrame, a y_column_name input and an x_column_names input input Your function should preprocess our data by performing:

mean subtraction from  y ,
dimension standardization for  x

In [ ]:
### GRADED

### YOUR ANSWER BELOW
def preprocess_for_regularization(data, y_column_name, x_column_names):
    """
    Perform mean subtraction and dimension standardization on data
        
    Positional argument:
        data -- a pandas dataframe of the data to pre-process
        y_column_name -- the name (string) of the column that contains
            the target of the training data.
        x_column_names -- a *list* of the names of columns that contain the
            observations to be standardized
        
    Returns:
        Return a DataFrame consisting only of the columns included
        in `y_column_name` and `x_column_names`.
        Where the y_column has been mean-centered, and the
        x_columns have been mean-centered/standardized.
        
        
    Example:
        data = pd.read_csv(tr_path).head()
        prepro_data = preprocess_for_regularization(data,'SalePrice', ['GrLivArea','YearBuilt'])
        
        print(prepro_data) #-->
                   GrLivArea  YearBuilt  SalePrice
                0  -0.082772   0.716753     7800.0
                1  -1.590161  -0.089594   -19200.0
                2   0.172946   0.657024    22800.0
                3  -0.059219  -1.911342   -60700.0
                4   1.559205   0.627159    49300.0
    
    NOTE: The sample standard deviation should be calculated with 0 "Delta Degrees of Freedom"
    
    If your answer does not match the example answer,
    check the default degrees of freedom in your standard deviation function.
    """
    return 

### BEGIN SOLUTION
def preprocess_for_regularization(data, y_column_name, x_column_names):
    
    # Create list of all columns
    toRetCol = x_column_names + [y_column_name]
    
    # subset dataframe
    toRet = data[toRetCol].copy() # For "setting with copy" warning
    
    # calculate mean of y, then subtract from all y's
    y_mean = np.mean(data[y_column_name])
    toRet[y_column_name] = data[y_column_name].apply(lambda x: x - y_mean)

    # Calc mean and std for every column in x
    # Then apply standardization
    for column in x_column_names:
        mean = np.mean(data[column])
        #std = data[column].std(ddof = 0)
        std = np.std(data[column])
        
        toRet[column] = data[column].apply(lambda x: (x - mean)/std)
    
    return toRet
### END SOLUTION

Next, you'll implement the equation for ridge regression using the closed form equation:

wRR=(λ+XTX)−1XTy
 
The function will be very similar to the function you wrote for Least Squares Regression with a slightly different matrix to invert.

NB: Many numpy matrix functions will be useful. e.g. np.matmul, np.linalg.inv, np.ones, np.transpose, and np.identity.

The main change from Least Squares Regression is that  λ  is a parameter we must set. This is different from the  w  parameters that we calculate from either closed form or approximation algorithms.

We will address tuning parameters such as  λ  in the next section.

Back to top

Question 6:
10 points

Code a function called "ridge_regression_weights" that takes, as input, three inputs: two matricies corresponding to the x inputs and y target and a number (int or float) for the lambda parameter

Your function should return a numpy array of regression weights

The following steps must be accomplished:

Ensure the number of rows of each the X matrix is greater than the number of columns. If not, transpose the matrix. Ultimately, the y input will have length n. Thus the x input should be in the shape n-by-p

Prepend an n-by-1 column of ones to the input_x matrix

Use the above equation to calculate the least squares weights. This will involve creating the lambda matrix - a p+1-by-p+1 matrix with the "lambda_param" on the diagonal

In [ ]:
### GRADED

### YOUR ANSWER BELOW

def ridge_regression_weights(input_x, output_y, lambda_param):
    """Calculate ridge regression least squares weights.
    
    Positional arguments:
        input_x -- 2-d matrix of input data
        output_y -- 1-d numpy array of target values
        lambda_param -- lambda parameter that controls how heavily
            to penalize large weight values
        
    Example:
        training_y = np.array([208500, 181500, 223500, 
                                140000, 250000, 143000, 
                                307000, 200000, 129900, 
                                118000])
                                
        training_x = np.array([[1710, 1262, 1786, 
                                1717, 2198, 1362, 
                                1694, 2090, 1774, 
                                1077], 
                               [2003, 1976, 2001, 
                                1915, 2000, 1993, 
                                2004, 1973, 1931, 
                                1939]])
        lambda_param = 10
        
        rrw = ridge_regression_weights(training_x, training_y, lambda_param)
        
        print(rrw) #--> np.array([-576.67947107,   77.45913349,   31.50189177])
        print(rrw[2]) #--> 31.50189177
        
    Assumptions:
        -- output_y is a vector whose length is the same as the
        number of observations in input_x
        -- lambda_param has a value greater than 0
    """
    
    weights = np.array([])
    return weights

### BEGIN SOLUTION
def ridge_regression_weights(input_x, output_y, lambda_param):
    
    # Check to ensure dataframe is long not wide
    if input_x.shape[0] < input_x.shape[1]:
        input_x = np.transpose(input_x)
    
    # Create column of ones
    ones = np.ones((len(output_y), 1), dtype=int)
    
    # Add column of ones to X
    augmented_x = np.concatenate((ones, input_x), axis=1)
    
    # Create square lambda_matrix, with size equal to number of columns in X
    lambda_matrix = lambda_param * np.identity(min(augmented_x.shape))
    
    # Invert lambda + dot-prod of x and transposed x
    inv = np.linalg.inv(lambda_matrix + np.matmul(np.transpose(augmented_x), augmented_x))
    
    # dot-prod of inverted matrix and transposed X
    left_multiplier = np.matmul(inv , np.transpose(augmented_x))
    
    # final dot-prod with the ys
    weights = np.matmul(left_multiplier, output_y)
    
    return weights
### END SOLUTION

Selecting the  λ  parameter
For our final function before looking at the sklearn implementation of ridge regression, we will create a hyperparameter tuning algorithm.

In ridge regression, we must pick a value for  λ . We have some intuition about  λ  from the equations that define it: small values tend to emulate the results from Least Squares, while large values will reduce the dimensionality of the problem. But the choice of  λ  can be motivated with a more precise quantitative treatment.

Eventually, we will look to choose the value of  λ  that minimizes the validation error, which we will determine using  k -fold cross-validation.

For this example, we will solve a simpler problem on finding a value that minimizes the list returned by the function.

In [ ]:
### Example of hiden function below:

### `hidden` takes a single number as a parameter (int or float) and returns a list of 1000 numbers
### the input must be between 0 and 50 exclusive

def hidden(hp):
    if (hp<=0) or (hp >= 50):
        print("input out of bounds")
    
    nums = np.logspace(0,5,num = 1000)
    vals = nums** 43.123985172351235134687934
    
    user_vals = nums** hp
    
    return vals-user_vals


In [ ]:
hidden(10)

In [ ]:
### GRADED

### YOUR ANSWER BELOW

def minimize( passed_func):
    """
    Find the numeric value that makes the mean of the
    output array returned from 'passed_func' as close to 0 as possible.
    
    Positional Argument:
        passed_func -- a function that takes a single number (between 0 and 50 exclusive)
            as input, and returns a list of 1000 floats.
        
    Example:
        passed_func = hidden
        min_hidden = minimize(passed_func)
        print(round(min_hidden,4))
        #--> 43.1204 (answers will vary slightly, must be close to 43.123985172351)
    
    """
    # Create values to test
    test_vals = ...
    
    # Find mean of returned array from function
    ret_vals = ...
    
    # Find smallest mean
    min_mean = ...
    
    # Return the test value that creates the smallest mean
    return ...

### BEGIN SOLUTION
def minimize(passed_func):
    
    # Create values to test
    test_vals = list(np.linspace(.1,49.9, 1000))
    
    # Find mean of returned array from function
    ret_vals = [abs(np.mean(passed_func(x))) for x in test_vals]
    
    # Find smallest mean
    min_mean = min(ret_vals)
    
    # Return the test value that creates the smallest mean
    return test_vals[ret_vals.index(min_mean)]
### END SOLUTION

The above simulates hyperparameter tuning.

In the case of ridge regression, you would be searching lambda parameters to minimize the validation error.

The hidden function would be analogous to the model building, the returned list would be analogous to the residuals, and the mean of that list would be analogous to the validation error.

See below for an example of using the functions built above that automatically perform hyperparameter tuning using mean absolute deviation.

In [ ]:
def lambda_search_func(lambda_param):
    
    # Define X and y
    # with preprocessing
    df = preprocess_for_regularization(data.head(50),'SalePrice', ['GrLivArea','YearBuilt'])
    
    y_true = df['SalePrice'].values
    X = df[['GrLivArea','YearBuilt']].values
    
    # Calculate Weights then use for predictions
    weights = ridge_regression_weights(X, y_true, lambda_param )
    y_pred = weights[0] + np.matmul(X,weights[1:])
    
    # Calculate Residuals
    resid = y_true - y_pred
    
    # take absolute value to tune on mean-absolute-deviation
    # Alternatively, could use:
    # return resid **2-S
    # for tuning on mean-squared-error
    
    return abs(resid)

minimize(lambda_search_func)    # --> about 2.9414414414414414

Question 8:
5 points

Why is cross-validation useful?

a) to minimize the liklihood of overfitting
b) Cross-validation allows us to fit on all our data
c) cross-validation standardizes outputs
d) cross-validation is not useful
Assing the character associated with your choice as a string to ans1

In [ ]:
### GRADED

### YOUR ANSWER BELOW

ans1 = None

### BEGIN SOLUTION
ans1 = 'a'
### END SOLUTION

Ridge Regression in sklearn
In the next question, we will ask you to implement Ridge regression in sklearn.

Back to top

Question 9:
10 points

Use the function LinearRegression from sklearn to instantiate the classifier lr. Use the function Ridge from sklear to instantiate the classifier reg. For this classifier, set the parameter alpha=100000. Use the Ridge function to instantiate another classifier, reg0, but, this time, set alpha=0.

Define a for loop with two indices, m and name. m will run over the three classifiers just defined and name will run over the list ["LeastSquares","Ridge alpha = 100000","Ridge, alpha = 0"]. In each iteration of your fol loop, you should fit the "X" ('GrLivArea' and 'YearBuilt') and the "y" ('SalePrice') from data.

Complete your for loop with the following print statement: print(name, "Intercept:", m.intercept_, "Coefs:",m.coef_,"\n")

NOTE: Note, the "alpha" parameter defines regularization strength. Lambda is a reserved word in Python -- Thus "alpha" instead

In [ ]:
## GRADED

from sklearn.linear_model import Ridge, LinearRegression

### YOUR ANSWER BELOW

lr = None
reg = None
reg0 = None
### BEGIN SOLUTION
### An alpha of 0 is equivalent to least-squares regression
lr = LinearRegression()
reg = Ridge(alpha = 100000)
reg0 = Ridge(alpha = 0)

# Notice how the consistent sklearn syntax may be used to easily fit many kinds of models
for m, name in zip([lr, reg, reg0], ["LeastSquares","Ridge alpha = 100000","Ridge, alpha = 0"]):
    
    m.fit(data[['GrLivArea','YearBuilt']], data['SalePrice'])
    print(name, "Intercept:", m.intercept_, "Coefs:",m.coef_,"\n")
### END SOLUTION

Note in the above example, an alpha of 100,000 is set for the ridge regularization. The reason an alpha value this high is required is because standardization/mean centering of our inputs did not occur, and instead of working with inputs on the order of [-4,4] we are on the interval of [0,2000].

Back to top

Question 10:
5 points

Above, the coefficent around 95/96 corresponds with:

a) Living Area
b) Year Built
c) Sale Price Assign character associated with your choice as string to ans2.

In [ ]:
### GRADED

### YOUR ANSWER BELOW

ans2 = None

### BEGIN SOLUTION
ans2 = 'a'
### END SOLUTION

Question 11:
5 points

True or False:

A larger "alpha" corresponds to a greater amount of regularization

Assign boolean choice to ans3

In [ ]:
### GRADED

### YOUR SOLUTION HERE
ans3 = None

### BEGIN SOLUTION
ans3 = True
### END SOLUTION